<img src="https://thevalley.es/wp-content/uploads/2016/11/thevalley-logo-negro.png" width="400"></img>

In [19]:
%%time 
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 




Leyendo lista de paquetes... Hecho
E: No se pudo abrir el fichero de bloqueo «/var/lib/apt/lists/lock» - open (13: Permiso denegado)
E: No se pudo bloquear el directorio /var/lib/apt/lists/
W: Se produjo un problema al desligar el fichero /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permiso denegado)
W: Se produjo un problema al desligar el fichero /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permiso denegado)
E: No se pudo abrir el fichero de bloqueo «/var/lib/dpkg/lock-frontend» - open (13: Permiso denegado)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
E: No se pudo abrir el fichero de bloqueo «/var/lib/dpkg/lock-frontend» - open (13: Permiso denegado)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
E: No se pudo abrir el fichero de bloqueo «/var/lib/dpkg/lock-frontend» - open (13: Permiso denegado)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
  Cl

In [18]:
!pip install --upgrade pyproj 
!pip install pygeos
!pip install --upgrade geopandas

  Using cached https://files.pythonhosted.org/packages/2c/12/7a8cca32506747c05ffd5c6ba556cf8435754af0939906cbcc7fa5802ea3/pyproj-3.0.1.tar.gz
  Installing build dependencies ... done
    Complete output from command python setup.py egg_info:
    ERROR: Minimum supported PROJ version is 7.2.0, installed version is 4.9.3. For more information see: https://pyproj4.github.io/pyproj/stable/installation.html
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-install-aghuxkaq/pyproj/
  Using cached https://files.pythonhosted.org/packages/d7/bf/e9cefb69d39155d122b6ddca53893b61535fa6ffdad70bf5ef708977f53f/geopandas-0.9.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c3/e2/00cacecafbab071c787019f00ad84ca3185952f6bb9bca9550ed83870d4d/pandas-1.1.5-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/9d/18/557d4f55453fe00f59807b111cc7b39ce53594e13ada88e16738fb4ff7f

# Carga de datos y descriptivos

El dataset que usaremos para el ejercicio corresponde a pisos para varias zonas de Madrid (nota: se ha generado en base a un modelo de valoración creado con información de portales inmobiliarios), los campos principales son:

* CODE codigo único del anuncio
* PRICE precio en euros
* AREA Superficie construida
* ROOMNUMBER Número de estancias (huecos)
* BATHNUMBER Número de baños
* STUDIO - Variable binaria  1 - Es un estudio 0 - No es un estudio
* ISPENTHHOUSE - Variable binaria  1 - Es un ático 0 - No es un ático
* DUPLEX - Variable binaria  1 - Es un dúplex 0 - No es un dúplex
* SWIMMINGPOOL - Variable binaria  1 - Tiene piscina - 0 No tiene piscina
* ELEVATOR - Variable binaria  1 - Tiene ascensor - 0 No tiene ascensor
* X - Coordenada geográfica X de la localización (longitud en EPGS:25830)
* Y - Coordenada geográfica X de la localización (latitud en EPGS:25830)

**Nota**: la proyección de las coordenadas es  la EPGS:25830 https://spatialreference.org/ref/epsg/etrs89-utm-zone-30n/ una unidad en esta medida representa un metro

## Descarga de datos y descriptivos básicos

Descargamos los datos desde el repositorio y extraemos algunos descriptivos

In [7]:
df_flats = pd.read_csv('https://raw.githubusercontent.com/davidreyblanco/ml-training/master/data/sample-flats-madrid.csv.gz', delimiter=';')
df_flats.describe()

,CODE,PRICE,AREA,ROOMNUMBER,BATHNUMBER,STUDIO,ISPENTHOUSE,DUPLEX,SWIMMINGPOOL,ELEVATOR,X,Y
count,2.733000e+03,2.733000e+03,2733.000000,2733.000000,2733.000000,2733.000000,2733.000000,2733.000000,2733.000000,2733.000000,2733.000000,2.733000e+03
mean,7.033958e+18,8.102975e+05,146.481888,2.935602,2.079034,0.019027,0.071716,0.027442,0.108306,0.903037,442558.737161,4.477613e+06
std,2.932817e+18,8.343778e+05,110.170071,1.478329,1.129423,0.136644,0.258064,0.163398,0.310823,0.295962,614.028907,6.612871e+02
min,6.034416e+15,4.828100e+04,22.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,441372.224765,4.476512e+06
25%,4.890341e+18,3.400020e+05,77.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,442120.762177,4.477055e+06
50%,9.223372e+18,5.252540e+05,108.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,442622.122753,4.477562e+06
75%,9.223372e+18,9.548910e+05,184.000000,4.000000,3.000000,0.000000,0.000000,0.000000,0.000000,1.000000,442998.347628,4.478111e+06
max,9.223372e+18,1.460009e+07,1170.000000,15.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,443872.278545,4.478966e+06


## Transformacion básica de variables

Generamos la variable de euros metro cuadrado y la llamaremos **UNITPRICE**, vamos a revisar que tenemos:

In [10]:
df_flats['UNITPRICE'] = df_flats['PRICE'] / df_flats['AREA']
df_flats['UNITPRICE'].describe()

count      2733.000000
mean       5388.578865
std        4544.193344
min         629.748503
25%        4089.204255
50%        4987.116279
75%        6039.840000
max      211595.449275
Name: UNITPRICE, dtype: float64

# Referencias
[K-Neighbors Scikitlearn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.neighbors)

# Ejercicios propuestos

## Clasificador de zonas gemelas
Tenemos un listado de zonas en el municipio de Madrid, cada zona tiene una serie de características numéricas, eliminamos del conjunto de entrada los barrios Sol, ....

## Prediciendo la felicidad

Podemos predecir la felicidad de un país en base a sus características, el ejercicio consiste en eliminar Japón, España, Alemania y Argelia del conjunto de entrenamiento y crear un modelo de k-vecinos de regresión con el resto de instancias. Una una vez lo tenemos tenemos que calcular la fiabilidad  de nuestro modelo con dos tres métricas:

* Error medio
* Error mediano
* Error cuadrático medio

El error lo calculamos como *nivel de felicidad predicho - nivel de felicidad real*

*Pregunta 1)* ¿Con que número k funciona mejor el algoritmo?

*Pregunta 2)*¿Cual es el efecto de usar pesos inversamente proporcionales a la distancia a no usarlos?